### 📘 Diagram 1 — FaceStats v3.5 High-Level Pipeline (Landscape)

```mermaid
flowchart LR
    A["📁 Raw Images<br>data/raw/"]
        --> B["🟦 Face Detection & Alignment<br>• InsightFace RetinaFace<br>• ONNXRuntime<br>• Resize + Crop<br><br><code>src/pipeline/preprocess.py</code>"]

    B --> C["🔷 Embedding Extraction (512-D)<br>• InsightFace ArcFace<br>• PyTorch backend<br>• L2-normalized embeddings<br><br><code>src/pipeline/embeddings.py</code>"]

    C --> D["💚 Attractiveness Model<br>• PyTorch MLP regressor<br>• torch.nn / torchvision<br><br><code>src/training/fs04_attractiveness_model.py</code><br><code>src/inference/attractiveness_infer.py</code>"]

    C --> E["🟫 Ethnicity Classifier (7-class)<br>• LightGBM<br>• scikit-learn LabelEncoder<br><br><code>src/training/fs08_ethnicity_model.py</code><br><code>src/inference/ethnicity_infer.py</code>"]

    C --> F["🟤 Age & Gender Prediction<br>• InsightFace Attribute Model<br>• ONNXRuntime<br><br><code>src/inference/age_gender_infer.py</code>"]

    D --> G
    E --> G
    F --> G

    G["📦 Checkpoint Writer<br>• Parquet / Arrow batching<br><br><code>data/interim/checkpoints/</code><br><code>src/pipeline/checkpoint_writer.py</code>"]
        --> H["🟨 Master Metadata Builder<br>• Polars + PyArrow<br>• Full merged dataset<br><br><code>src/pipeline/metadata_merge.py</code>"]

    H --> I["🟩 Composite Generator<br>• Mean Face & PCA<br>• NumPy / OpenCV<br><br><code>src/pipeline/composites.py</code>"]

    H --> J["📊 Visualization Dashboards<br>• Streamlit UI<br>• Matplotlib / Seaborn<br><br><code>src/visualization/app.py</code>"]
```


### 📘 Diagram 2 — Data Flow Diagram (Landscape)

```mermaid
flowchart LR
    A["📁 Raw Images<br>data/raw/"]
        --> B["🟦 Preprocessed Faces<br>aligned + cropped<br><code>data/interim/preprocessed/</code>"]

    B --> C["🔷 Embeddings (512-D)<br>ArcFace<br><code>data/processed/embeddings/</code>"]

    C --> D["🧠 ML Predictions<br>• Attractiveness<br>• Ethnicity<br>• Age/Gender"]

    D --> E["📦 Checkpoint Chunks<br>Parquet/Arrow<br><code>data/interim/checkpoints/</code>"]

    E --> F["🟨 Merged Metadata<br>fs00_master.parquet<br><code>data/processed/</code>"]

    F --> G["🟩 Composite Images<br>Mean faces, PCA<br><code>data/processed/composites/</code>"]

    F --> H["📊 Visual Analytics<br>Streamlit Dashboard"]
```


### 📘 Diagram 3 — Attractiveness Model Architecture (Landscape)

```mermaid
flowchart LR

    A["512-D Face Embedding"]
        --> B["Linear 512→256<br>ReLU + Dropout"]

    B --> C["Linear 256→128<br>ReLU + Dropout"]

    C --> D["Linear 128→64<br>ReLU"]

    D --> E["Linear 64→1<br>📈 Score (1–10)"]

    E --> F["Loss: MSE<br>Optimizer: Adam"]
```


### 📘 Diagram 4 — Ethnicity Classifier (Landscape)

```mermaid
flowchart LR

    A["Input: 512-D Embedding"]
        --> B["LightGBM Booster<br>• Multi-class (7)<br>• ~1500 trees<br>• depth=7"]

    B --> C["Softmax Outputs<br>7-class probabilities"]

    C --> D["Label Decoding<br>Inverse LabelEncoder"]

    D --> E["Final Prediction<br>+ full probability vector"]
```


### 📘 Diagram 5 — Age & Gender Attribute Model (Landscape)

```mermaid
flowchart LR

    A["Aligned 112×112 Face"]
        --> B["InsightFace Attribute Network<br>ONNXRuntime"]

    B --> C["Regression Head<br>Age (float)"]

    B --> D["Classification Head<br>Gender (M/F)"]

    C --> E["Age Output"]
    D --> F["Gender Output"]
```


### 📘 Diagram 6 — Checkpoint & Storage Layout (Landscape)

```mermaid
flowchart LR

    A["Raw Images<br>data/raw/"]
        --> B["Preprocessed<br>data/interim/preprocessed/"]

    B --> C["Embeddings<br>data/processed/embeddings/"]

    C --> D["Checkpoint Chunks<br>Arrow/Parquet<br>chunk_0.parquet … chunk_N.parquet"]

    D --> E["Merged Metadata File<br>fs00_master.parquet"]

    E --> F["Composite Images<br>data/processed/composites/"]

    E --> G["Streamlit Dashboard<br>visualization"]
```


### 📘 Diagram 7 — Multiprocessing Execution Map (Landscape)

```mermaid
flowchart LR

    A["Main Controller<br>fs07_multiprocessing.py"]
        --> B["Worker Pool<br>torch.multiprocessing<br>N workers"]

    B --> C["Worker Process<br>• Loads models once<br>• Runs embedding + predictions<br>• Emits result dict"]

    C --> D["Checkpoint Writer<br>batch → Parquet/Arrow"]
```


### 📘 Diagram 8 — Composite Generator (Landscape)

```mermaid
flowchart LR

    A["Select aligned faces<br>(N samples)"]
        --> B["Resize + Stack Arrays<br>NumPy float tensors"]

    B --> C["Mean Face<br>pixel-wise average"]

    C --> D["PCA (optional)<br>eigencomponents"]

    D --> E["Save Composite PNG<br>and composite.json metadata"]
```


### 📘 Diagram 9 — User Upload → Score Flow (Landscape)

```mermaid
flowchart LR

    A["User Uploads Image"]
        --> B["Face Detection + Alignment<br>RetinaFace (ONNX)"]

    B --> C["Embedding Extraction (512-D)<br>ArcFace / PyTorch"]

    C --> D["Attractiveness (MLP)"]
    C --> E["Ethnicity (LightGBM)"]
    C --> F["Age + Gender (InsightFace)"]

    D --> G["JSON Output"]
    E --> G
    F --> G
```


## 📦 Idealized Repository Structure

```text
FaceStats_v3.5/
├── README.md
├── requirements.txt
├── LICENSE
│
├── config/
│   ├── settings.yaml                     # Global config (paths, thresholds, model names)
│   ├── model_paths.yaml                  # ONNX/PyTorch model paths
│   ├── preprocessing.yaml                # Resize/crop settings
│   └── app_config.yaml                   # Streamlit / dashboard settings
│
├── data/
│   ├── raw/                              # Raw images (input)
│   ├── interim/
│   │   ├── preprocessed/                 # Aligned/cropped faces
│   │   ├── checkpoints/                  # chunk_*.parquet (embedding + predictions)
│   │   └── temp/                         # Temporary multiprocessing batch cache
│   ├── processed/
│   │   ├── embeddings/                   # 512-D ArcFace embeddings
│   │   ├── metadata/                     # fs00_master.parquet + merged metadata
│   │   └── composites/                   # Output composite faces
│   └── models/                           # Trained model artifacts (MLP, LightGBM)
│
├── notebooks/
│   ├── schematics.ipynb                  # All diagrams + architectural flowcharts
│   ├── exploration/                      # EDA / distribution plots
│   ├── training_attractiveness.ipynb     # MLP training
│   ├── training_ethnicity.ipynb          # LightGBM training
│   ├── inspect_embeddings.ipynb          # Sanity checks
│   └── demo_user_upload.ipynb            # Upload → score demo
│
├── src/
│   ├── pipeline/
│   │   ├── preprocess.py                 # Face detection + alignment (RetinaFace ONNX)
│   │   ├── embeddings.py                 # ArcFace 512-D embedding extractor
│   │   ├── checkpoint_writer.py          # Writes parquet/arrow batches
│   │   ├── metadata_merge.py             # Merges all checkpoints → fs00_master.parquet
│   │   ├── composites.py                 # Mean face + PCA composite generator
│   │   └── orchestrator.py               # Runs entire pipeline end-to-end
│   │
│   ├── inference/
│   │   ├── attractiveness_infer.py       # MLP regressor inference
│   │   ├── ethnicity_infer.py            # LightGBM ethnicity inference
│   │   ├── age_gender_infer.py           # InsightFace attribute ONNX inference
│   │   └── user_upload_api.py           # Wrapper for “upload → score”
│   │
│   ├── training/
│   │   ├── attractiveness_model.py       # PyTorch MLP model class
│   │   ├── train_attractiveness.py       # Full training loop (loss curves)
│   │   ├── ethnicity_model.py            # LightGBM training + encoder save
│   │   ├── train_ethnicity.py            # Training script
│   │   └── metrics.py                    # Loss curves, accuracy plots
│   │
│   ├── visualization/
│   │   ├── app.py                        # Streamlit main dashboard
│   │   ├── plots.py                      # Matplotlib/Seaborn visual helpers
│   │   └── compare_embeddings.py         # Heatmaps, PCA scatter, similarity
│   │
│   ├── multiprocessing/
│   │   ├── workers.py                    # Worker process logic
│   │   ├── job_manager.py                # Multi-worker orchestration
│   │   └── batch_scheduler.py            # Distributes image batches
│   │
│   └── utils/
│       ├── file_io.py                    # Path helpers, safe load/save
│       ├── image_ops.py                  # CV2/Numpy helpers
│       ├── logging_utils.py              # Colored rich logging
│       ├── timer.py                      # Profiling decorators
│       └── seed.py                       # Deterministic seeding
│
└── tests/
    ├── test_preprocess.py                # Face detection tests
    ├── test_embeddings.py                # Embedding shape + L2 norm
    ├── test_inference.py                 # Attractiveness/Ethnicity/Age-Gender sanity
    ├── test_metadata_merge.py            # Merge correctness tests
    └── test_composites.py                # Mean face generation tests
```
